In [9]:
# Fit the mixed effects model
from patsy import dmatrix
from statsmodels.formula.api import mixedlm
from statsmodels.stats.multicomp import MultiComparison
import pandas as pd
import numpy as np
from pymer4.models import Lmer

pd.options.mode.chained_assignment = None
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_columns", 5000)
pd.set_option("display.width", 10000)

In [10]:
data = pd.read_csv("./dataset_copy_DELETEAFTER.csv")

metric="mean"
input_id="benchmark"
system_id="acquisition"
bin_id="budget"
bins=[0,15,30,49]


differentMeans_model = mixedlm(formula=f"{metric} ~ {system_id}", data=data, groups=input_id)
diffModelFit = differentMeans_model.fit( reml=False)
print(diffModelFit.summary())
print(diffModelFit.random_effects)

bins_set = set(bins)
bins_set.add(data[bin_id].min())
bins_set.add(data[bin_id].max())
bins = sorted(list(bins_set))

bin_labels = [f"{bins[i]}_{bins[i+1]}" for i in range(len(bins) - 1)]


data[f"{bin_id}_bins"] = pd.cut(
    data[bin_id], bins=bins, labels=bin_labels, include_lowest=True
)

# New model "expanded": Divides into system AND bin-classes (Term system:bin_id allows for Cartesian Product, i.e. different Mean for each system and bin-class)
model_expanded = Lmer(
    f"{metric} ~  {system_id} + {bin_id}_bins + {system_id}:{bin_id}_bins + (1 | {input_id})",
    data=data,
)
model_expanded.fit(factors={
    system_id: list(data[system_id].unique()),
    f"{bin_id}_bins": list(data[f"{bin_id}_bins"].unique())},
REML=False,
summarize=False,
)
#print(model_expanded.ranef)
#print("")
#print(model_expanded.summary())


                       Mixed Linear Model Regression Results
Model:                     MixedLM         Dependent Variable:         mean        
No. Observations:          180000          Method:                     ML          
No. Groups:                4               Scale:                      53.4669     
Min. group size:           45000           Log-Likelihood:             -613546.8042
Max. group size:           45000           Converged:                  Yes         
Mean group size:           45000.0                                                 
-----------------------------------------------------------------------------------
                                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------
Intercept                                 5.760    4.374  1.317 0.188 -2.813 14.332
acquisition[T.ProbabilityOfImprovement]   0.337    0.073  4.609 0.000  0.194  0.480
acquisition[T.U

In [11]:
# Get predicted values for each level of system_id
"""
grid = (
    np.array(
        np.meshgrid(
            data[input_id].unique(),
            data[system_id].unique(),
        )
    )
    .reshape(2, len(data[input_id].unique()) * len(data[system_id].unique())).T)

grid = pd.DataFrame(grid, columns=[input_id, system_id)"""
grid = (
    np.array(
        np.meshgrid(
            data[system_id].unique(),
        )
    )
    .reshape(1, len(data[system_id].unique())).T)
grid = pd.DataFrame(grid, columns=[ system_id])
print("Grid:\n",grid)
betas = diffModelFit.fe_params
print("Coeffs:\n",betas)
mat = dmatrix(f"C({system_id})", grid, return_type="matrix")
print("Matrix:\n",mat)
emmeans = grid.copy()
emmeans["means"] = mat @ betas
#print(emmeans)
vcov = diffModelFit.cov_params()
# print(vcov)

vcov = vcov[~vcov.index.str.contains("Var|Cor")]
vcov = vcov.loc[:, ~vcov.columns.str.contains("Var|Cor")]
#print(vcov)
emmeans["SE"] = np.sqrt(np.diagonal(mat @ vcov) @ mat.T)
print(emmeans)


Grid:
                  acquisition
0        ExpectedImprovement
1   ProbabilityOfImprovement
2       UpperConfidenceBound
3       qExpectedImprovement
4         qKnowledgeGradient
5  qProbabilityOfImprovement
6              qSimpleRegret
7      qUpperConfidenceBound
8               randomSearch
Coeffs:
 Intercept                                   5.759636
acquisition[T.ProbabilityOfImprovement]     0.336997
acquisition[T.UpperConfidenceBound]         0.118736
acquisition[T.qExpectedImprovement]         0.061894
acquisition[T.qKnowledgeGradient]           0.495266
acquisition[T.qProbabilityOfImprovement]    0.523160
acquisition[T.qSimpleRegret]                0.096439
acquisition[T.qUpperConfidenceBound]        0.152831
acquisition[T.randomSearch]                 0.892227
dtype: float64
Matrix:
 [[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0.

In [31]:
grid[input_id]=data[input_id].unique()[0]
#print("Grid:\n",grid)
predicted_values = diffModelFit.predict(grid)
#print("Predicted values:\n",predicted_values)
#print(pd.DataFrame(predicted_values, columns=["pred"]))
# Perform Tukey's HSD test

means=emmeans["means"]
standard_errors=emmeans["SE"]
sample_sizes=20
group_labels=data[system_id].unique()
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.stats.libqsturng as tukey
tr=pairwise_tukeyhsd(means,  group_labels, alpha=0.05)#pairwise_tukeyhsd(means, group_labels)
#print(tr)

tukey_results = MultiComparison(predicted_values, grid[system_id]).tukeyhsd(
    alpha=0.05
)

# calculate the standard deviation for each pair of groups
#print(tukey_results.std_pairs)
#print(tukey_results.summary())
#print("tukey end")

import statsmodels
from scipy.stats import t


# Calculate the standard errors for each group
SE = emmeans["SE"]

n=len(emmeans[system_id])
# Calculate the pooled standard deviation
SD = SE/(n**2)
MSE=SD
print(MSE)
# Calculate the q-statistic for each pairwise comparison
q_stats = []
for i in range(n):
    for j in range(i,n):
        if i != j:
            d = float(emmeans.iloc[i]["means"]) - float(emmeans.iloc[j]["means"])
            q = np.sqrt(2) * abs(d) / s
            q_stats.append(q)

# Calculate the critical value
alpha = 0.05
df = 179996
critical_value = t.ppf(1 - alpha / 2, df)

# Compare the q-statistics to the critical value
p_values = []
for q in q_stats:
    if q <= critical_value:
        p = 1 - t.cdf(q, df)
        p_values.append(p)
    else:
        p_values.append(p)
        #p_values.append(np.nan)
print(len(p_values))
print(p_values)
# Print the results in a pairwise manner
c=0
for i in range(number_of_samples):
    for j in range(i,number_of_samples):
        c+=1
        if i != j:
            print(f'{emmeans.iloc[i]["means"] -emmeans.iloc[j]["means"]}: q={q_stats[c]}, p={p_values[c]}')

0    0.053999
1    0.054003
2    0.054003
3    0.054003
4    0.054003
5    0.054003
6    0.054003
7    0.054003
8    0.054003
Name: SE, dtype: float64
36
[0.1950384500081659, 0.03517582347057269, 0.17275811008774866, 0.17275811008774866, 0.17275811008774866, 0.07080627745719625, 0.07080627745719625, 0.07080627745719625, 0.07080627745719625, 0.07080627745719625, 0.07080627745719625, 0.07080627745719625, 0.07080627745719625, 0.07080627745719625, 0.07080627745719625, 0.19315623833882523, 0.19315623833882523, 0.19315623833882523, 0.3669918180963192, 0.3016547026095453, 0.3016547026095453, 0.3016547026095453, 0.3016547026095453, 0.2992726887748903, 0.08287821755113056, 0.08287821755113056, 0.33537157015466157, 0.33537157015466157, 0.33537157015466157, 0.33537157015466157, 0.33537157015466157, 0.33537157015466157, 0.33537157015466157, 0.1950384500081659, 0.1950384500081659, 0.1950384500081659]
-0.3369966713463821: q=0.9433244880176012, p=0.17275811008774866
-0.118735837727308: q=7.5483406067

c:\Users\Amega\Git\significance_analysis\.venv_3_10_0\lib\site-packages\numpy\core\fromnumeric.py:3747: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\Amega\Git\significance_analysis\.venv_3_10_0\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


IndexError: list index out of range